In [8]:
import pandas as pd
from bs4 import BeautifulSoup as bs
from splinter import Browser
import re
import time

In [9]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [10]:
info_url = "https://space-facts.com/mars/"
news_url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
twitter_url = "https://twitter.com/marswxreport?lang=en"
hemisphere_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

In [11]:
browser.visit(news_url)
html = browser.html
soup = bs(html, 'html.parser')
titles = soup.find_all('div', class_='content_title')
paras = soup.find_all('div', class_='article_teaser_body')
first_title = titles[1].text
first_para = paras[0].text

In [12]:
browser.visit(image_url)
html = browser.html
soup = bs(html, 'html.parser')
images = soup.find_all('article', class_='carousel_item')
image_link = "https://www.jpl.nasa.gov" + images[0].find('a')['data-fancybox-href']

In [14]:
browser.visit(twitter_url)
time.sleep(1)
html = browser.html
soup = bs(html, 'html.parser')
pattern = re.compile('InSight')
tweets = soup.find_all(text=pattern)
tweets[0]

'InSight sol 521 (2020-05-15) low -93.2ºC (-135.7ºF) high -1.1ºC (30.0ºF)\nwinds from the SW at 4.7 m/s (10.4 mph) gusting to 18.3 m/s (40.8 mph)\npressure at 7.00 hPa'

In [29]:
tables = pd.read_html(info_url)
mars_df = tables[0]
mars_df.columns = ['Category', 'Value']
mars_df.set_index('Category', inplace=True)
html_table = mars_df.to_html()
html_table = html_table.replace('\n', '')

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Value</th>    </tr>    <tr>      <th>Category</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

In [19]:
browser.visit(hemisphere_url)
html = browser.html
soup = bs(html, 'html.parser')
hemisphere_titles = soup.find_all('div', class_='description')
hemisphere_info = []
for title in hemisphere_titles:
    hemisphere_info.append({'title': title.find('h3').text})
    for item in hemisphere_info:
        item['title'] = item['title'].replace(' Enhanced', '')

In [20]:
for item in hemisphere_info:
    browser.visit(hemisphere_url)
    html = browser.html
    soup = bs(html, 'html.parser')
    browser.click_link_by_partial_text(item['title'])
    html = browser.html
    soup = bs(html, 'html.parser')
    hemisphere_pics = soup.find_all('div', class_='downloads')
    item['img_url'] = hemisphere_pics[0].find('a')['href']

C:\Users\nitec\Anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [24]:
browser.quit()